# Analyzing Used Car Listings on eBay Kleinanzeigen¶


We will be working on a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle.The data dictionary provided with data is as follows:

   * dateCrawled - When this ad was first crawled. All field-values are taken from this date.
   * name - Name of the car.
   * seller - Whether the seller is private or a dealer.
   * offerType - The type of listing
   * price - The price on the ad to sell the car.
   * abtest - Whether the listing is included in an A/B test.
   * vehicleType - The vehicle Type.
   * yearOfRegistration - The year in which which year the car was first registered.
   * gearbox - The transmission type.
   * powerPS - The power of the car in PS.
   * model - The car model name.
   * kilometer - How many kilometers the car has driven.
   * monthOfRegistration - The month in which which year the car was first registered.
   * fuelType - What type of fuel the car uses.
   * brand - The brand of the car.
   * notRepairedDamage - If the car has a damage which is not yet repaired.
   * dateCreated - The date on which the eBay listing was created.
   * nrOfPictures - The number of pictures in the ad.
   * postalCode - The postal code for the location of the vehicle.
   * lastSeenOnline - When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings.

In [1]:
import pandas as pd
import numpy as np

In [37]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

In [39]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Dataset contains 20 columns, most of which are stored as strings. There are a few columns with null values, but no columns have more than ~20% null values. There are some columns that contain dates stored as strings.

We'll start by cleaning the column names to make the data easier to work with.

### Cleaned Columns

We'll make a few changes here:

* Change the columns from camelcase to snakecase.
* Change a few wordings to more accurately describe the columns.

In [40]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [41]:
autos.columns = ['data_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest', 'vehicle_type', 
                 'registration_year', 'gear_box', 'power_PS', 'model', 'kilometer', 'registration_month', 'fuel_type', 'brand',
                 'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code', 'last_seen']
autos.head()

,data_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gear_box,power_PS,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


### Initial Data Exploration and Cleaning

We'll start by exploring the data to find obvious areas where we can clean the data.



In [42]:
autos.describe(include='all')

,data_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gear_box,power_PS,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,333659,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,338142,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,8,NaN,2,NaN,251,NaN,NaN,7,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:45:59
freq,7,657,371525,371516,NaN,192585,95894,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


Our initial observations:

There are a number of text columns where all (or nearly all) of the values are the same:
* seller
* offer_type

The num_photos column looks odd, we'll need to investigate this further.

In [43]:
autos["num_photos"].value_counts()

0    371528
Name: num_photos, dtype: int64

It looks like the num_photos column has 0 for every column. We'll drop this column, plus the other two we noted as mostly one value.

In [44]:
autos["seller"].value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

In [45]:
autos["offer_type"].value_counts()

Angebot    371516
Gesuch         12
Name: offer_type, dtype: int64

In [46]:
autos = autos.drop(["seller","num_photos","offer_type"], axis=1)

In [54]:
autos.rename({'kilometer':'kilometer_drove'}, axis=1, inplace=True)
autos.head()

,data_crawled,name,price,abtest,vehicle_type,registration_year,gear_box,power_PS,model,kilometer_drove,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,60437,2016-04-06 10:17:21


### Exploring Kilometer and Prices

In [55]:
autos["kilometer_drove"].value_counts()

150000    240797
125000     38067
100000     15920
90000      12523
80000      11053
70000       9773
60000       8669
50000       7615
5000        7069
40000       6376
30000       6041
20000       5676
10000       1949
Name: kilometer_drove, dtype: int64

We can see that the values in this field are rounded, which might indicate that sellers had to choose from pre-set options for this field. 
Additionally, there are more high mileage than low mileage vehicles.



In [60]:
print(autos["price"].unique().shape)
print(autos["price"].describe(include='all'))
autos["price"].value_counts().head(20)

(5597,)
count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64


0       10778
500      5670
1500     5394
1000     4649
1200     4594
2500     4438
600      3819
3500     3792
800      3784
2000     3432
999      3364
750      3203
650      3150
4500     3053
850      2946
2200     2936
700      2936
1800     2886
900      2874
950      2793
Name: price, dtype: int64

Again, the prices in this column seem rounded, however given there are 5597 unique values in the column, that may just be people's tendency to round prices on the site.

There are 10778 cars listed with 0 price - given that this is only 2% of the of the cars, we might consider removing these rows. The maximum price is one hundred million Euros, which seems a lot, let's look at the highest prices further.

In [61]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

2147483647     1
99999999      15
99000000       1
74185296       1
32545461       1
27322222       1
14000500       1
12345678       9
11111111      10
10010011       1
10000000       8
9999999        3
3895000        1
3890000        1
2995000        1
2795000        1
1600000        2
1300000        1
1250000        2
1234566        1
Name: price, dtype: int64

In [62]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
11        5
12        8
13        7
14        5
15       27
16        2
17        5
18        3
19        3
20       51
Name: price, dtype: int64

There are a number of listings with prices below Euro 30, including about 1,500 at Euro 0. There are also a small number of listings with very high values, including 14 at around or over 1 Euro million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is Euro 1. We will keep the \$1 items, but remove anything above Euro 350,000, since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [63]:
autos = autos[autos["price"].between(1, 351000)]
autos["price"].describe()

count    360635.000000
mean       5898.671956
std        8866.359669
min           1.000000
25%        1250.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

###  Exploring the date columns¶


There are a number of columns with date information:

* date_crawled
* registration_month
* registration_year
* ad_created
* last_seen
* These are a combination of dates that were crawled, and dates with meta-information from the crawler. The non-registration   dates are stored as strings.

We'll explore each of these columns to learn more about the listings.

In [64]:
autos[['data_crawled','ad_created','last_seen']][0:5]

,data_crawled,ad_created,last_seen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21


In [69]:
(autos["data_crawled"].str[:10].value_counts(normalize=True, dropna=False).sort_index())

2016-03-05    0.025547
2016-03-06    0.014483
2016-03-07    0.035657
2016-03-08    0.033469
2016-03-09    0.034115
2016-03-10    0.032645
2016-03-11    0.032773
2016-03-12    0.036242
2016-03-13    0.015783
2016-03-14    0.036330
2016-03-15    0.033424
2016-03-16    0.030205
2016-03-17    0.031647
2016-03-18    0.013119
2016-03-19    0.035271
2016-03-20    0.036400
2016-03-21    0.035682
2016-03-22    0.032493
2016-03-23    0.032002
2016-03-24    0.029914
2016-03-25    0.032800
2016-03-26    0.031974
2016-03-27    0.030227
2016-03-28    0.035063
2016-03-29    0.034126
2016-03-30    0.033535
2016-03-31    0.031872
2016-04-01    0.034145
2016-04-02    0.035094
2016-04-03    0.038812
2016-04-04    0.037628
2016-04-05    0.012780
2016-04-06    0.003128
2016-04-07    0.001617
Name: data_crawled, dtype: float64

In [71]:
(autos["data_crawled"].str[:10].value_counts(normalize=True, dropna=True).sort_values())

2016-04-07    0.001617
2016-04-06    0.003128
2016-04-05    0.012780
2016-03-18    0.013119
2016-03-06    0.014483
2016-03-13    0.015783
2016-03-05    0.025547
2016-03-24    0.029914
2016-03-16    0.030205
2016-03-27    0.030227
2016-03-17    0.031647
2016-03-31    0.031872
2016-03-26    0.031974
2016-03-23    0.032002
2016-03-22    0.032493
2016-03-10    0.032645
2016-03-11    0.032773
2016-03-25    0.032800
2016-03-15    0.033424
2016-03-08    0.033469
2016-03-30    0.033535
2016-03-09    0.034115
2016-03-29    0.034126
2016-04-01    0.034145
2016-03-28    0.035063
2016-04-02    0.035094
2016-03-19    0.035271
2016-03-07    0.035657
2016-03-21    0.035682
2016-03-12    0.036242
2016-03-14    0.036330
2016-03-20    0.036400
2016-04-04    0.037628
2016-04-03    0.038812
Name: data_crawled, dtype: float64

Looks like the site was crawled daily over roughly a one month period in March and April 2016. The distribution of listings crawled on each day is roughly uniform.

In [72]:
(autos["last_seen"].str[:10].value_counts(normalize=True, dropna=False).sort_index())

2016-03-05    0.001264
2016-03-06    0.004098
2016-03-07    0.005202
2016-03-08    0.007939
2016-03-09    0.009824
2016-03-10    0.011460
2016-03-11    0.012955
2016-03-12    0.023240
2016-03-13    0.008410
2016-03-14    0.012176
2016-03-15    0.016324
2016-03-16    0.016418
2016-03-17    0.028699
2016-03-18    0.006888
2016-03-19    0.016330
2016-03-20    0.019884
2016-03-21    0.020026
2016-03-22    0.020508
2016-03-23    0.018015
2016-03-24    0.019163
2016-03-25    0.019000
2016-03-26    0.015958
2016-03-27    0.016721
2016-03-28    0.022189
2016-03-29    0.023284
2016-03-30    0.023725
2016-03-31    0.024243
2016-04-01    0.023897
2016-04-02    0.024967
2016-04-03    0.025308
2016-04-04    0.025536
2016-04-05    0.126962
2016-04-06    0.218950
2016-04-07    0.130437
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [74]:
print(autos["ad_created"].unique().shape)
((autos["ad_created"].str[:10].value_counts(normalize=True, dropna=False).sort_index()))

(114,)


2014-03-10    0.000003
2015-03-20    0.000003
2015-06-11    0.000003
2015-06-18    0.000003
2015-08-07    0.000003
2015-08-10    0.000003
2015-09-04    0.000006
2015-09-09    0.000003
2015-10-14    0.000003
2015-11-02    0.000003
2015-11-08    0.000003
2015-11-10    0.000003
2015-11-12    0.000003
2015-11-13    0.000003
2015-11-17    0.000003
2015-11-23    0.000006
2015-11-24    0.000006
2015-12-05    0.000008
2015-12-06    0.000003
2015-12-17    0.000003
2015-12-27    0.000003
2015-12-28    0.000003
2015-12-30    0.000008
2016-01-02    0.000011
2016-01-03    0.000003
2016-01-06    0.000003
2016-01-07    0.000008
2016-01-08    0.000003
2016-01-10    0.000011
2016-01-13    0.000008
                ...   
2016-03-09    0.034162
2016-03-10    0.032520
2016-03-11    0.032806
2016-03-12    0.036078
2016-03-13    0.017056
2016-03-14    0.035266
2016-03-15    0.033350
2016-03-16    0.030427
2016-03-17    0.031245
2016-03-18    0.014022
2016-03-19    0.034303
2016-03-20    0.036488
2016-03-21 

There is a large variety of ad created dates. Most fall within 1-2 months of the listing date, but a few are quite old, with the oldest at around 9 months.

In [75]:
autos["registration_year"].describe()

count    360635.000000
mean       2004.433133
std          81.016977
min        1000.000000
25%        1999.000000
50%        2004.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

The year that the car was first registered will likely indicate the age of the car. Looking at this column, we note some odd values. The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future.

### Dealing with Incorrect Registration Year Data¶


Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:

In [112]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

0.038751091824143526

Given that this is less than 4% of our data, we will remove these rows.



In [113]:
# Many ways to select rows in a dataframe that fall within a value range for a column.
# Using `Series.between()` is one way.
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.066699
1999    0.063552
2005    0.062669
2006    0.057708
2001    0.056955
2003    0.056557
2004    0.056173
2002    0.054290
2007    0.050499
1998    0.049691
Name: registration_year, dtype: float64

It appears that most of the vehicles were first registered in the past 20 years.



### Exploring Price by brand

In [78]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.213557
bmw               0.108570
opel              0.107394
mercedes_benz     0.095981
audi              0.088685
ford              0.068892
renault           0.048331
peugeot           0.029975
fiat              0.025963
seat              0.018947
skoda             0.015462
mazda             0.015373
smart             0.014347
citroen           0.014036
nissan            0.013595
toyota            0.012822
hyundai           0.009960
sonstige_autos    0.009511
mini              0.009309
volvo             0.008979
mitsubishi        0.008224
honda             0.007581
kia               0.006924
suzuki            0.006314
alfa_romeo        0.006278
porsche           0.006025
chevrolet         0.004975
chrysler          0.003829
dacia             0.002468
jeep              0.002182
daihatsu          0.002149
land_rover        0.002105
subaru            0.002085
jaguar            0.001686
daewoo            0.001478
saab              0.001439
trabant           0.001384
l

German manufacturers represent four out of the top five brands, almost 50% of the overall listings. Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.

There are lots of brands that don't have a significant percentage of listings, so we will limit our analysis to brands representing more than 5% of total listings.

In [84]:
brand_counts = autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [100]:
brand_mean_prices = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_prices = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_prices)    
brand_mean_prices

{'volkswagen': 5322,
 'bmw': 8361,
 'opel': 2941,
 'mercedes_benz': 8462,
 'audi': 8963,
 'ford': 3660}

Of the top 5 brands, there is a distinct price gap:

   * Audi, BMW and Mercedes Benz are more expensive
   * Ford and Opel are less expensive
   * Volkswagen is in between - this may explain its popularity, it may be a 'best of 'both worlds' option.

### Exploring Milage

In [101]:
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series, columns=['mean_milage'])

,mean_milage
volkswagen,5322
bmw,8361
opel,2941
mercedes_benz,8462
audi,8963
ford,3660


In [102]:
brand_mean_milage = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_milage = brand_only["kilometer_drove"].mean()
    brand_mean_milage[brand] = int(mean_milage)
    
brand_mean_milage

{'volkswagen': 128614,
 'bmw': 132920,
 'opel': 128873,
 'mercedes_benz': 130670,
 'audi': 129687,
 'ford': 123896}

In [103]:
mean_milage = pd.Series(brand_mean_milage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

In [104]:
brand_info = pd.DataFrame(mean_milage, columns=["mean_milage"])
brand_info

,mean_milage
bmw,132920
mercedes_benz,130670
audi,129687
opel,128873
volkswagen,128614
ford,123896


In [109]:
brand_info["mean_prices"] = mean_prices

brand_info

,mean_milage,mean_prices
bmw,132920,8361
mercedes_benz,130670,8462
audi,129687,8963
opel,128873,2941
volkswagen,128614,5322
ford,123896,3660


The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.